<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference:

<big> for quicker and effortless model downloads

In [ ]:
#@title Check GPU
!nvidia-smi

In [ ]:
#@title Install Dependencies
from IPython.display import clear_output
from ipywidgets import Button




# ===== Clone Repository ===== #

!git clone -b master https://github.com/ArkanDash/Advanced-RVC-Inference

%cd  /content/Advanced-RVC-Inference



# ===== Install requirements ===== #
!pip install pip==24.0
!pip install -r ./requirements.txt

In [ ]:
#@title Download Hubert & base Model




!python ./models.py

In [ ]:
#@title youtube downloader for separation
import yt_dlp
import os

def download_wav(video_url, output_dir):
  ydl_opts = {
      'format': 'bestaudio/best',
      'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
      'noplaylist': True,  # Download only a single video, not a playlist
      'extract_audio': True,
      'audio_format': 'wav', # Ensure WAV format
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }]
  }

  try:
      with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          ydl.download([video_url])
          print(f"Download completed. Audio saved in {output_dir}")
  except yt_dlp.utils.DownloadError as e:
      print(f"An error occurred: {e}")

# Example usage
video_url = 'Video_URL' #@param {type:"string"}
 # Replace with your YouTube video URL
output_directory = '/content/yt_dlp' #@param {type:"string"}
 # customize your output directory

# Create the output directory if it doesn't exist.
os.makedirs(output_directory, exist_ok=True)

download_wav(video_url, output_directory)

In [ ]:
#@title audio separator for infernece



from audio_separator.separator import Separator

aud_input = "Input_aud"  #@param {type:"string"}

output_dir = "/content/separated"  #@param {type:"string"}

# Initialize the Separator class (with optional configuration properties, below)
separator = Separator(output_dir=output_dir, output_format="wav")

separator.load_model()

output_names = {
    "Vocals": "vocals_output",
    "Instrumental": "instrumental_output",
}
output_files = separator.separate(aud_input, output_names)
print(f"Separation complete!")

In [ ]:




#@title ## Inference
import os
main_dir = "/content/Harmonify"
os.chdir(main_dir)
from lib.infer import infer_audio
from google.colab import files
from pydub import AudioSegment
import shutil

MODEL_NAME = "" #@param {type:"string"}
SOUND_PATH = "" #@param {type:"string"}
F0_CHANGE = 0 #@param {type:"integer"}
F0_METHOD = "fcpe" #@param ["crepe", "harvest", "mangio-crepe", "rmvpe", "rmvpe+", "fcpe", "fcpe_legacy", "hybrid[mangio-crepe+rmvpe]", "hybrid[mangio-crepe+fcpe]", "hybrid[rmvpe+fcpe]", "hybrid[mangio-crepe+rmvpe+fcpe]"] {allow-input:true}
CREPE_HOP_LENGTH = 120 #@param {type:"integer"}
INDEX_RATE = 0.75 #@param {type:"number"}
FILTER_RADIUS = 3 #@param {type:"integer"}
RMS_MIX_RATE = 0.25 #@param {type:"number"}
PROTECT = 0.33 #@param {type:"number"}
SPLIT_INFER = False #@param {type:"boolean"}
MIN_SILENCE = 500 #@param {type:"number"}
SILENCE_THRESHOLD = -50 #@param {type:"number"}
SEEK_STEP = 1 #@param {type:"slider", min:1, max:10, step:1}
KEEP_SILENCE = 200 #@param {type:"number"}
QUEFRENCY = 0 #@param {type:"number"}
TIMBRE = 1 #@param {type:"number"}
F0_AUTOTUNE = False #@param {type:"boolean"}
OUTPUT_FORMAT = "wav" #@param ["wav", "flac", "mp3"]



inferred_audio = infer_audio(
    MODEL_NAME,
    SOUND_PATH,
    F0_CHANGE,
    F0_METHOD,
    CREPE_HOP_LENGTH,
    INDEX_RATE,
    FILTER_RADIUS,
    RMS_MIX_RATE,
    PROTECT,
    SPLIT_INFER,
    MIN_SILENCE,
    SILENCE_THRESHOLD,
    SEEK_STEP,
    KEEP_SILENCE,
    QUEFRENCY,
    TIMBRE,
    F0_AUTOTUNE,
    OUTPUT_FORMAT
)
os.chdir(main_dir)

print(f"Showing {inferred_audio}.")
AudioSegment.from_file(inferred_audio)